# A short presentation showing speedup when switching from pandas to FireDucks

🔥 🐦[**FireDucks**](https://fireducks-dev.github.io/) is a high-performance compiler-accelerated DataFrame library with highly compatible pandas APIs developed to speedup a pandas application without any manual source code changes. It comes with a multi-threaded C++ kernel and automatic query optimization features (powered by an in-built compiler) with lazy-execution model.

In this test drive, we will be using [ Parking Violations Issued - Fiscal Year 2022](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/7mxj-7a6y/about_data) dataset from NYC Open Data.

REF: https://colab.research.google.com/drive/12tCzP94zFG2BRduACucn5Q_OcX1TUKY3




In [ ]:
!pip install fireducks

In [ ]:
#import os
#os.environ["FIREDUCKS_FLAGS"] = "--benchmark-mode"

In [ ]:
%load_ext fireducks.pandas
import time
import pandas as pd

In [ ]:
print(f"evaluation with {pd.__name__} version: {pd.__version__}")

evaluation with fireducks.pandas version: 0.12.4


In [ ]:
## Let's load the parquet dataset

In [ ]:
# Data can be downloaded from here:
#!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

In [ ]:
# this method enforces the execution of the compiled IRs for the input frame.
# use it for FireDucks to measure the actual computation time
def evaluate(df):
  if hasattr(df, "_evaluate"):
    df._evaluate()

In [ ]:
t0 = time.time()
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description",
             "Vehicle Body Type", "Issue Date", "Summons Number"]
)
evaluate(df)
load_t = time.time() - t0
df

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
0,NY,<NA>,VAN,06/25/2021,1457617912
1,NY,<NA>,SUBN,06/25/2021,1457617924
2,TX,<NA>,SDN,06/17/2021,1457622427
3,MO,<NA>,SDN,06/16/2021,1457638629
4,NY,<NA>,TAXI,07/04/2021,1457639580
...,...,...,...,...,...
15435602,99,21-No Parking (street clean),SUBN,06/07/2022,8995222761
15435603,TN,21-No Parking (street clean),PICK,06/07/2022,8995222773
15435604,NY,21-No Parking (street clean),2DSD,06/07/2022,8995222785
15435605,VA,21-No Parking (street clean),SUBN,06/07/2022,8995222827


## Q1: Which parking violation is most commonly committed by vehicles from various U.S states?

In [ ]:
t1 = time.time()
r1 = (df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)
evaluate(r1)
q1_t = time.time() - t1
r1

,Registration State,Violation Description,count
0,99,<NA>,17550
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
62,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
63,WA,21-No Parking (street clean),3732
64,WI,14-No Standing,1639
65,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


## Q2: Which vehicle body types are most frequently involved in parking violations?

In [ ]:
t2 = time.time()
r2 = (df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)
evaluate(r2)
q2_t = time.time() - t2
r2

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
YANT,1
YBSD,1
YEL,1


## Q3. How do parking violations vary across days of the week?

In [ ]:
t3 = time.time()
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)
r3 = df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()
evaluate(r3)
q3_t = time.time() - t3
r3

issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

## Evaluation

In [ ]:
s = pd.Series([load_t, q1_t, q2_t, q3_t], index = ["data_loading", "query_1", "query_2", "query_3"])
print(f"total time taken: {s.sum()} sec")
s

total time taken: 7.584502696990967 sec


data_loading    3.324290
query_1         0.592974
query_2         0.409812
query_3         3.257427
dtype: float64

📢 Apart from the explicit evaluation using _evaluate() method, you may also like to activate the benchmark-mode by enabling the commented code in **Cell#2** and restart and re-execute this notebook. This time the lazy-execution mode will be disabled, so each API will be executed right after it is called (as in pandas).

For the examples in this notebook there were not enough room for compiler optimization, hence you may not notice significant difference in execution time even when lazy-execution mode is disabled.

Here is my finding from the exeution when I tried:

*   Native pandas: **15 sec**
*   FireDucks.pandas without benchmark-mode (multithreaded + compiler optimized): **7.5 sec**



# Conclusion

🚀 Execution time could be reduced **from 15 sec to 7.5 sec (~2x speedup)** even for an execution environment with low spec (default colab environment seems to support only 2 cpu cores). So without incurring any migration cost (pandas to FireDucks code translation is absolutely not required) or an expensive hardware cost (no need for high spec system), you can enjoy faster analysis with FireDucks!!

🚀🚀 You may like to check other [benchmarks](https://fireducks-dev.github.io/docs/benchmarks/)